#### MSA search process directly reuses ColabFold


refer to https://github.com/sokrypton/ColabFold?tab=readme-ov-file

In [ ]:
# Install miniforge
# https://github.com/conda-forge/miniforge#download
conda create -n colabfold_search python=3.10
conda activate colabfold_scratch
cd ~/ && mkdir workdir && cd workdir

In [ ]:
git clone https://github.com/soedinglab/MMseqs2.git
cd MMseqs2
git checkout 71dd32ec43e3ac4dabf111bbc4b124f1c66a85f1 # As in colabfold readme
mkdir build
cd build
cmake -DCMAKE_BUILD_TYPE=RELEASE -DCMAKE_INSTALL_PREFIX=. ..
make
make install 
export PATH=$(pwd)/bin/:$PATH
cd ../

In [ ]:
git clone https://github.com/sokrypton/ColabFold
cd ColabFold 
git checkout 9bd39fd8be88ca71c039be9a8000bb0db92f80a2 
# download and prepare the database
# Note: Uniref only version uniref30_2103 has contained taxonomy information
# you can download it from https://wwwuser.gwdg.de/~compbiol/colabfold/uniref30_2103_taxonomy.tar.gz
bash setup_databases.sh 
# If you want to add Taxonomy ID using uniref30_2103 database
# refer to https://github.com/sokrypton/ColabFold/issues/216
# Add the following line after Line 93 in colabfold/mmseqs/search.py
# run_mmseqs(mmseqs, ["convertalis", base.joinpath("qdb"), dbbase.joinpath(f"{uniref_db}{dbSuffix1}"),
#                     base.joinpath("res_exp_realign_filter"), base.joinpath("uniref_tax.m8"), "--format-output",
#                     "query,target,taxid,taxname,taxlineage,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,cigar",
#                     "--db-load-mode", str(db_load_mode), "--threads", str(threads)])

pip3 install .
pip install colabfold[alphafold]==1.5.5

In [ ]:
export PYTHONPATH=$PYTHONPATH:ColabFold
mkdir -p logs
mkdir -p results
python3 ColabFold/colabfold/mmseqs/search.py --mmseqs=mmseqs {input_file} {db_dir} {output_dir} --db1 uniref30_2103_db --db2 pdb70_220313_db --db3 colabfold_envdb_202108_db --use-templates 1 --db-load-mode 2 --threads 32
# if you use scripts/msa/data/pdb_seqs/pdb_seq.fasta as {input_file}
# you will get the following results as in scripts/msa/data/mmcif_msa_initial
# 0.a3m  1.a3m  2.a3m  3.a3m  pdb70_220313_db.m8  uniref_tax.m8

#### The above MSA Pipeline is how we generate the MSA data for training.
- It depends on specific versions of ColabFold and MMseq.
- It requires modifying the ColabFold code and using a specific version of Uniref30 to generate MSA with taxonomy information.
- The overall solution is to search the MSA containing taxonomy information only once for the unique sequence, and pair it according to the species information of each MSA.

